In [1]:
from elasticsearch import Elasticsearch #, helpers
from datetime import datetime
from pprint import pprint
from dateutil.parser import parse

import json
import pandas as pd
import csv

In [2]:
tweets_data = pd.read_json('./boulder_flood_geolocated_tweets.json', lines=True)

In [3]:
tweets_data

,created_at,id,id_str,text,truncated,entities,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,...,favorite_count,favorited,retweeted,possibly_sensitive,lang,extended_entities,retweeted_status,quoted_status_id,quoted_status_id_str,quoted_status
0,2013-12-31 07:14:22+00:00,417916626596806656,417916626596806656,Boom bitch get out the way! #drunk #islands #g...,False,"{'hashtags': [{'text': 'drunk', 'indices': [28...","<a href=""http://instagram.com"" rel=""nofollow"">...",NaN,NaN,NaN,...,0,False,False,0.0,en,NaN,NaN,NaN,NaN,NaN
1,2013-12-31 18:49:31+00:00,418091565161017345,418091565161017344,@WeatherDude17 Not that revved up yet due to m...,False,"{'hashtags': [{'text': 'snow', 'indices': [108...","<a href=""https://about.twitter.com/products/tw...",4.180914e+17,4.180914e+17,1.214464e+09,...,0,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN
2,2013-12-30 20:29:20+00:00,417754295455723520,417754295455723520,Story of my life! 😂 #boulder http://t.co/ZMfNK...,False,"{'hashtags': [{'text': 'boulder', 'indices': [...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,...,0,False,False,0.0,en,"{'media': [{'id': 417754295334088704, 'id_str'...",NaN,NaN,NaN,NaN
3,2013-12-30 23:02:29+00:00,417792838428925952,417792838428925952,We're looking for the two who came to help a c...,False,"{'hashtags': [{'text': 'Boulder', 'indices': [...","<a href=""http://www.tweetcaster.com"" rel=""nofo...",NaN,NaN,NaN,...,0,False,False,0.0,en,NaN,NaN,NaN,NaN,NaN
4,2014-01-01 06:12:15+00:00,418263379027820544,418263379027820544,Happy New Year #Boulder !!!! What are some of ...,False,"{'hashtags': [{'text': 'Boulder', 'indices': [...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,...,0,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18816,2013-09-12 04:12:20+00:00,378008148759420928,378008148759420928,RT @EricRobertson: A more representative flood...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","<a href=""http://www.twitter.com"" rel=""nofollow...",NaN,NaN,NaN,...,0,False,False,0.0,en,NaN,{'created_at': 'Thu Sep 12 03:54:00 +0000 2013...,NaN,NaN,NaN
18817,2013-09-12 04:03:47+00:00,378005998872432640,378005998872432640,Warning. Flash flood of boulder creek is immin...,False,"{'hashtags': [{'text': 'boulderflood', 'indice...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,...,0,False,False,NaN,en,NaN,NaN,NaN,NaN,NaN
18818,2013-09-12 04:11:56+00:00,378008050335875074,378008050335875072,I just got an “Emergency Alert”! Nice to know ...,False,"{'hashtags': [{'text': 'cowx', 'indices': [112...","<a href=""http://tapbots.com/tweetbot"" rel=""nof...",NaN,NaN,NaN,...,0,False,False,0.0,en,"{'media': [{'id': 378008050017136640, 'id_str'...",NaN,NaN,NaN,NaN
18819,2013-09-12 04:05:59+00:00,378006549735538688,378006549735538688,RT @dailycamera: RT @CUBoulderPolice: Some fl...,False,"{'hashtags': [{'text': 'CUBoulder', 'indices':...","<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,NaN,...,0,False,False,NaN,en,NaN,{'created_at': 'Thu Sep 12 04:05:17 +0000 2013...,NaN,NaN,NaN


In [4]:
tweets_data.shape

(18821, 29)

In [5]:
tweets_data = tweets_data[tweets_data['coordinates'].notna()]

In [6]:
tweets_data.shape

(3943, 29)

In [7]:
tweets_data['created_at'] = pd.to_datetime(tweets_data['created_at'])

C:\Users\DEll\AppData\Local\Temp/ipykernel_1920/1947022294.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_data['created_at'] = pd.to_datetime(tweets_data['created_at'])


In [8]:
mapping={
  "mappings": {
      "properties": {
          "text": {
              "type": "text"
          },
          "created_at": {
              "type": "date",
          },
          "location": {  
              "type": "geo_point"
      }
      }
  }
}

In [29]:
es = Elasticsearch()

index_name = 'tweets'

if es.indices.exists(index_name):
    es.indices.delete(index=index_name)

es.indices.create(index=index_name, ignore=400, body=mapping)

C:\Users\DEll\AppData\Local\Temp/ipykernel_1920/2012908906.py:5: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  if es.indices.exists(index_name):
C:\Users\DEll\AppData\Local\Temp/ipykernel_1920/2012908906.py:8: DeprecationWarning: The 'body' parameter is deprecated for the 'create' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  es.indices.create(index=index_name, ignore=400, body=mapping)


{'acknowledged': True, 'shards_acknowledged': True, 'index': 'tweets'}

In [10]:
tweets_list = tweets_data.values.tolist()

In [11]:
for sample in tweets_list :
    tweet = {
      "created_at": sample[0],
      "text": sample[3],
      "location": sample[14]['coordinates']
    }
    res = es.index(index=index_name, body=tweet)

C:\Users\DEll\AppData\Local\Temp/ipykernel_1920/4113600160.py:7: DeprecationWarning: The 'body' parameter is deprecated for the 'index' API and will be removed in a future version. Instead use the 'document' parameter. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  res = es.index(index=index_name, body=tweet)
C:\ProgramData\Anaconda3\lib\site-packages\elasticsearch\connection\base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


In [ ]:
qQuery = {
      "query": {
        "bool": {
          "must": {
            "match_all": {}
          },
          "filter": {
            "geo_bounding_box": {
              "location": {
                "top_left": [144.98467167, -37.80312131],
                "bottom_right": [-105.27725101, 40.01544423]
              }
            }
          }
        }
      }
    }

In [25]:
'''
qQuery2 = {
  "query": {
    "bool": {
      "must": {
        "match_all": {}
      },
      "filter": [
          {
            "match": {
                "created_at": '2013-12-31T07:14:22+00:00'
            }
          },
          {
            "match": {
                "text": "Boom"
            }
          },
          {
              "geo_bounding_box": {
                  "location": {
                      "top_left": [144.98467167, -37.80312131],
                      "bottom_right": [-105.27725101, 40.01544423]
                  }
          }
        }
      ]
    }
  }
}
'''

In [24]:
'''
qQuery3 = {
  "query": {
    "bool": {
      "must": {
        "match_all": {}
      },
      "filter": [
        {
          "range": {
              "created_at": {
                  "gte": '2013-09-19T11:42:03+00:00', 
                  "lte": '2013-12-31T07:14:22+00:00'
              }
          }
        },
        {
          "match": {
            "text": "Boom"
          }
        },
        {
          "geo_bounding_box": {
              "location": {
                  "top_left": [144.98467167, -37.80312131],
                  "bottom_right": [-105.27725101, 40.01544423]
          }
        }}
      ]
    }
  }
}
'''

In [ ]:
response = es.search(
    index=index_name,
    query=qQuery
)

for hit in response['hits']['hits']:
    print(hit["_source"]) 

In [14]:
text_query = {
  "query": {
            "match": {
                "text": "Boom"
            }
  }
}

In [15]:
text_res = es.search(
    index=index_name,
    body=text_query
)

for hit in text_res['hits']['hits']:
    pprint(hit["_source"]) 

C:\Users\DEll\AppData\Local\Temp/ipykernel_1920/3739189330.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  text_res = es.search(


{'created_at': '2013-12-31T07:14:22+00:00',
 'location': [-118.10041174, 34.14628356],
 'text': 'Boom bitch get out the way! #drunk #islands #girlsnight  #BJs '
         '#hookah #zephyrs #boulder #marines… http://t.co/uYmu7c4o0x'}
{'created_at': '2013-09-19T11:42:03+00:00',
 'location': [-105.08423572, 40.00246529],
 'text': 'Perversely, when I heard a boom and saw a flash this AM, I hoped it '
         'was an explosion, not thunder, lightning &amp; rain. #cowx '
         '#lafayetteco'}


In [16]:
date_query = {
  "query": {
            "match": {
                "created_at": '2013-12-31T07:14:22+00:00'
            }
  }
}

In [17]:
date_res = es.search(
    index=index_name,
    body=date_query
)

#print("Got %d Hits:" % res['hits']['total']['value'])
for hit in date_res['hits']['hits']:
    pprint(hit["_source"]) #     print("%(timestamp)s %(author)s: %(text)s" % hit["_source"])

{'created_at': '2013-12-31T07:14:22+00:00',
 'location': [-118.10041174, 34.14628356],
 'text': 'Boom bitch get out the way! #drunk #islands #girlsnight  #BJs '
         '#hookah #zephyrs #boulder #marines… http://t.co/uYmu7c4o0x'}


C:\Users\DEll\AppData\Local\Temp/ipykernel_1920/3537882960.py:1: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  date_res = es.search(


In [22]:
loc_query = {
  "query": {
            "match": {
                "location": [144.98467167, -37.80312131],
          }
  }
}

In [ ]:
loc_res = es.search(
    index=index_name,
    body=loc_query
)

for hit in loc_res['hits']['hits']:
    pprint(hit["_source"])